In [1]:
import pandas as pd
import numpy

In [2]:
lg_train = pd.read_csv('/home/student/Notebooks/lg_train.csv')
lg_test = pd.read_csv('/home/student/Notebooks/lg_test.csv')
rf_train = pd.read_csv('rf_train.csv')
rf_test = pd.read_csv('rf_test.csv')
xgb_train = pd.read_csv('xgb_train.csv')
xgb_test = pd.read_csv('xgb_test.csv')
cat_train = pd.read_csv('cat_train.csv')
cat_test = pd.read_csv('cat_test.csv')
lgb_train = pd.read_csv('lbm_train.csv')
lgb_test = pd.read_csv('lbm_test.csv')

In [3]:
train = pd.read_csv('/SharedHDD/Code/ML/Recidivism Project/NIJ_s_Recidivism_Challenge_Training_Dataset.csv')
train = train[(train['Recidivism_Arrest_Year1']==False) & (train['Recidivism_Arrest_Year2']==False)]
target  = train['Recidivism_Arrest_Year3'].astype(int)

In [4]:
l1_train = pd.concat([lg_train, rf_train, xgb_train, cat_train, lgb_train], axis=1)
l1_test = pd.concat([lg_test, rf_test, xgb_test, cat_test, lgb_test ], axis=1)

In [5]:
l1_train

,lg,rf_1,rf_2,rf_3,xgb_1,xgb_2,xgb_3,xgb_4,xgb_5,cat_1,cat_2,cat_3,cat_4,cat_5,lbm_1,lbm_2,lbm_3,lbm_4,lbm_5
0,0.204402,0.203536,0.209946,0.205718,0.244947,0.267102,0.250593,0.246028,0.242137,0.191431,0.178625,0.183114,0.180203,0.159950,0.209170,0.219742,0.225284,0.206741,0.206202
1,0.254664,0.300665,0.292147,0.302126,0.372659,0.373607,0.352222,0.382433,0.376186,0.263556,0.335578,0.319955,0.328867,0.283396,0.318786,0.359416,0.362772,0.322799,0.328796
2,0.219878,0.240752,0.239924,0.239561,0.156298,0.209846,0.193065,0.204964,0.198593,0.247618,0.248610,0.253798,0.272038,0.194988,0.236056,0.219357,0.227147,0.245442,0.238583
3,0.135663,0.195898,0.189723,0.186438,0.186392,0.206614,0.184763,0.177670,0.178682,0.186207,0.207345,0.202315,0.195446,0.203682,0.194401,0.189151,0.191156,0.187452,0.185251
4,0.048433,0.056613,0.063436,0.054096,0.003930,0.009657,0.003706,0.001842,0.001581,0.017314,0.023211,0.016322,0.008345,0.025072,0.012217,0.015088,0.017824,0.012605,0.012551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9393,0.053065,0.158967,0.144794,0.157813,0.087400,0.091636,0.094358,0.089836,0.091667,0.098586,0.099804,0.091630,0.104027,0.098291,0.119370,0.109809,0.122508,0.123399,0.119764
9394,0.043584,0.133506,0.133115,0.131738,0.090163,0.094970,0.100127,0.092482,0.100651,0.084655,0.089675,0.074680,0.089494,0.081539,0.060763,0.061864,0.061869,0.065360,0.060984
9395,0.050743,0.057505,0.057852,0.056067,0.065776,0.073505,0.079582,0.066372,0.069683,0.053075,0.047779,0.058853,0.056246,0.056324,0.067911,0.068720,0.063863,0.075426,0.071552
9396,0.031175,0.057597,0.058824,0.056864,0.043347,0.050855,0.043858,0.034267,0.034686,0.062928,0.064422,0.064515,0.070449,0.061400,0.055179,0.057628,0.053825,0.058687,0.056966


In [6]:
from scipy.special import expit
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import RidgeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import pandas
random_state = 2021
n_folds = 5
k_fold = StratifiedKFold(n_splits=n_folds, random_state=random_state, shuffle=True)

train_preds = numpy.zeros(len(l1_train.index), )
test_preds = numpy.zeros(len(l1_test.index), )
features = l1_train.columns

for fold, (train_index, test_index) in enumerate(k_fold.split(l1_train, target)):
    print("--> Fold {}".format(fold + 1))
    y_train = target.iloc[train_index]
    y_valid = target.iloc[test_index]

    x_train = pandas.DataFrame(l1_train[features].iloc[train_index])
    x_valid = pandas.DataFrame(l1_train[features].iloc[test_index])
    
    model = CalibratedClassifierCV(
        RidgeClassifier(random_state=random_state), 
        cv=3
    )
    model.fit(
        x_train,
        y_train,
    )

    train_oof_preds = model.predict_proba(x_valid)[:,-1]
    test_oof_preds = model.predict_proba(l1_test[features])[:,-1]
    train_preds[test_index] = train_oof_preds
    test_preds += test_oof_preds / n_folds
    print(": ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds, average="micro")))
    print("")
    
print("--> Overall metrics")
print(": ROC AUC Score = {}".format(roc_auc_score(target, train_preds, average="micro")))

--> Fold 1
: ROC AUC Score = 0.6842933071010652

--> Fold 2
: ROC AUC Score = 0.6956224902546635

--> Fold 3
: ROC AUC Score = 0.7003748816476478

--> Fold 4
: ROC AUC Score = 0.712619233891258

--> Fold 5
: ROC AUC Score = 0.7073136241593483

--> Overall metrics
: ROC AUC Score = 0.6996037253588979


In [7]:
test_preds

array([0.10099774, 0.26027653, 0.22424408, ..., 0.09388634, 0.10358601,
       0.06104437])

In [8]:
l1_train.describe()

,lg,rf_1,rf_2,rf_3,xgb_1,xgb_2,xgb_3,xgb_4,xgb_5,cat_1,cat_2,cat_3,cat_4,cat_5,lbm_1,lbm_2,lbm_3,lbm_4,lbm_5
count,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000,9398.000000
mean,0.190856,0.190892,0.191360,0.190878,0.189391,0.189839,0.189627,0.189760,0.189452,0.190827,0.189625,0.190730,0.190988,0.190605,0.188842,0.189348,0.189168,0.189296,0.189101
std,0.104681,0.079312,0.077313,0.079308,0.116973,0.109381,0.110418,0.115556,0.113242,0.099378,0.096616,0.099078,0.092988,0.094615,0.097259,0.097226,0.097246,0.094325,0.095239
min,0.016783,0.011653,0.015797,0.012037,0.000069,0.001302,0.000660,0.000220,0.000183,0.001720,0.003279,0.000935,0.001816,0.001819,0.003409,0.004332,0.006189,0.003506,0.003359
25%,0.110833,0.132585,0.133290,0.132985,0.101955,0.109038,0.107693,0.103837,0.105679,0.118524,0.117784,0.118222,0.124755,0.121845,0.115310,0.114900,0.113832,0.117452,0.116737
50%,0.171832,0.195085,0.194362,0.195321,0.177911,0.181208,0.181136,0.179370,0.179407,0.185070,0.185413,0.186063,0.187072,0.186100,0.181520,0.182436,0.181993,0.182976,0.181881
75%,0.250005,0.245439,0.245800,0.245849,0.259285,0.257865,0.257089,0.259595,0.258478,0.253856,0.253465,0.255247,0.250820,0.251403,0.252915,0.254874,0.253836,0.252711,0.252277
max,0.768694,0.472724,0.469771,0.482994,0.759605,0.700998,0.691247,0.735187,0.753605,0.654448,0.604228,0.651865,0.637490,0.591291,0.598406,0.582848,0.580481,0.611558,0.612481


In [9]:
test = pd.read_csv('/SharedHDD/Code/ML/Recidivism Project/NIJ_s_Recidivism_Challenge_Test_Dataset3.csv')

In [10]:
test['Probability'] = test_preds

In [11]:
test['Probability']

0       0.100998
1       0.260277
2       0.224244
3       0.235759
4       0.105545
          ...   
4141    0.082473
4142    0.061113
4143    0.093886
4144    0.103586
4145    0.061044
Name: Probability, Length: 4146, dtype: float64

In [12]:
test = test[['ID', 'Probability']]
test['Probability'] = test['Probability'].round(4)
test.to_csv('/SharedHDD/Code/ML/Recidivism Project/EconCGU_3YearForecast.csv', index = False)

In [13]:
test['Probability']

0       0.1010
1       0.2603
2       0.2242
3       0.2358
4       0.1055
         ...  
4141    0.0825
4142    0.0611
4143    0.0939
4144    0.1036
4145    0.0610
Name: Probability, Length: 4146, dtype: float64

In [14]:
target.value_counts()

0    7607
1    1791
Name: Recidivism_Arrest_Year3, dtype: int64